In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image, ImageOps

from RGBcubical_utils import compute_RGB_contributions, difference_RGB_ECP, approximate_difference_RGB_ECP

from tqdm.notebook import tqdm

# load images

In [ ]:
images = []

images.append(Image.open("data/textures/easy/banded/banded_{}.png".format(0)))

images.append(Image.open("data/textures/easy/chessboard/chessboard_{}.png".format(0)))
    
np_images = [np.int16(img) for img in images]

fig, axs = plt.subplots(1, 2, figsize=(10, 5))

for i in range(2):
    axs[i].imshow(np_images[i])

In [ ]:
def add_noise(input_matrix, noise_min = -25, noise_max = +25, p = 0.1, seed=42):
    
    np.random.seed(seed)

    noise_matrix = np.random.randint(low=noise_min, high=noise_max, 
                                     size=input_matrix.shape, dtype=int)
    
    # we want to add noise only to p fraction of pixels
    sampling_matrix = np.random.uniform(low=0.0, high=1.0, size=input_matrix.shape)
    sampling_matrix[sampling_matrix <= p] = 1
    sampling_matrix[sampling_matrix < 1] = 0
    sampling_matrix = np.int16(sampling_matrix)
    
    return(np.clip( input_matrix + np.multiply(noise_matrix, sampling_matrix),
                    a_min=0, a_max=255))

In [ ]:
NUM_SAMPLES = 10

noisy_images = []

for im in np_images:
    noisy_images += [add_noise(im, seed=i, noise_min=-10, noise_max=10, p=1)
                     for i in range(NUM_SAMPLES)]

fig, axs = plt.subplots(4, 5,
                       figsize=(15, 10))

for i in range(5):
    axs[0, i].imshow(noisy_images[i])
    axs[1, i].imshow(noisy_images[i+5])
    axs[2, i].imshow(noisy_images[i+10])
    axs[3, i].imshow(noisy_images[i+15])

# Compute RGB Euler profiles

In [ ]:
list_of_RGB_contributions = [compute_RGB_contributions(img) for img in tqdm(noisy_images)]

# Distance matrix

In [ ]:
for c in list_of_RGB_contributions:
    print(len(c))

In [ ]:
%%time
distance_matrix_RBG = np.zeros((len(list_of_RGB_contributions), 
                                len(list_of_RGB_contributions)))

for i in tqdm(range(len(list_of_RGB_contributions))):
    for j in tqdm(range(i+1, len(list_of_RGB_contributions))):
        distance_matrix_RBG[i,j] = difference_RGB_ECP(list_of_RGB_contributions[i], list_of_RGB_contributions[j],
                                                 inf_value=256)
        distance_matrix_RBG[j,i] = distance_matrix_RBG[i,j]

In [ ]:
%%time
approx_distance_matrix = np.zeros((len(list_of_RGB_contributions), 
                                   len(list_of_RGB_contributions)))

for i in tqdm(range(len(list_of_RGB_contributions))):
    for j in tqdm(range(i+1, len(list_of_RGB_contributions))):
        approx_distance_matrix[i,j] = approximate_difference_RGB_ECP(list_of_RGB_contributions[i], 
                                                                     list_of_RGB_contributions[j],
                                                                    inf_value=256)
        approx_distance_matrix[j,i] = approx_distance_matrix[i,j]

In [ ]:
# assert(np.array_equal(approx_distance_matrix, distance_matrix))

In [ ]:
distance_matrix_RBG = approx_distance_matrix

im0 = plt.imshow(distance_matrix_RBG)
plt.colorbar(im0)
plt.title('RGB')
plt.show()

In [ ]:
distance_matrix_RBG[1]

# Compute Euler Characteristic

In [ ]:
grayscale_images = [ImageOps.grayscale(img) for img in noisy_images]
np_grayscale_images = [np.expand_dims(np.int16(img), axis=2) for img in grayscale_images]

In [ ]:
from pyEulerCurves import plot_euler_curve, difference_ECC

In [ ]:
# given the ordered list of local contributions
# returns a list of tuples (filtration, euler characteristic)
def euler_characteristic_list_from_all(local_contributions):

    euler_characteristic = []
    old_f, current_characteristic = local_contributions[0]

    for filtration, contribution in local_contributions[1:]:
        if filtration > old_f:
            euler_characteristic.append([old_f, current_characteristic])
            old_f = filtration

        current_characteristic += contribution

    # add last contribution
    if len(local_contributions) > 1:
        euler_characteristic.append([filtration, current_characteristic])

    return euler_characteristic

In [ ]:
list_of_ECC_contributions = [ [ (c[0][0], c[1]) for c in compute_RGB_contributions(img)] 
                             for img in tqdm(np_grayscale_images)] 

In [ ]:
list_of_ECC = [euler_characteristic_list_from_all(contrib) for contrib in list_of_ECC_contributions]

In [ ]:
for c in list_of_ECC_contributions:
    print(len(c))

In [ ]:
fig, axs = plt.subplots(4, 5,
                       figsize=(15, 10))

for i in range(5):
    axs[0, i].plot_euler_curve(list_of_ECC[i],    axs[0, i], with_lines=True)
    axs[1, i].plot_euler_curve(list_of_ECC[i+5],  axs[1, i], with_lines=True)
    axs[2, i].plot_euler_curve(list_of_ECC[i+10], axs[2, i], with_lines=True)
    axs[3, i].plot_euler_curve(list_of_ECC[i+15], axs[3, i], with_lines=True)

for i in range(NUMBER_OF_SAMPLES):
    plot_euler_curve(list_of_ECC[i], axs[i], with_lines=True)

In [ ]:
distance_matrix_GRAY = np.zeros((len(list_of_RGB_contributions), 
                                   len(list_of_RGB_contributions)))
for i in tqdm(range(len(list_of_ECC))):
    for j in range(i+1, len(list_of_ECC)):
        distance_matrix_GRAY[i,j] = difference_ECC(list_of_ECC[i], list_of_ECC[j], max_f = 255)
        distance_matrix_GRAY[j,i] = difference_ECC(list_of_ECC[j], list_of_ECC[i], max_f = 255)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(6.4*2, 4.8))

im0 = axs[0].imshow(distance_matrix_RBG)
plt.colorbar(im0, ax=axs[0])
axs[0].set_title('RGB')


im1 = axs[1].imshow(distance_matrix_GRAY)
plt.colorbar(im1, ax=axs[1], vmin=0
axs[1].set_title('Grayscale')

plt.show()